In [24]:
#### NAPRAVIO SAM VELIKE IZMENE U OVOJ SVESCI!!! NE KORISTITI !!!

import sys
import os
sys.path.append('../')
from data_loader import DataLoader
import random
from feature_selector import BaselineSelector
from feature_selector import AnalysisSelector
from feature_selector import FeatureExtractor
from utils import parse_config_features
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import configparser
import re

#### all channels

In [25]:
config = configparser.ConfigParser()
config.read('../config.ini')

# all features of one epoch
X = pd.read_csv(os.path.join('..', config['Data']['features_path']))
# labels of one epoch
y = np.load(os.path.join('..', config['Data']['labels_path']))
# participant ids of one epoch
participants = np.load(os.path.join('..', config['Data']['par_ids_path']))

# drop ids
X = X.drop(X.columns[0], axis=1)
# append participants labels to find means per person
X['participant'] = participants
X['label'] = y

In [26]:
means = X.groupby(['participant', 'label']).mean()
means

,,skewness_alpha_0,skewness_alpha_1,skewness_alpha_2,skewness_alpha_3,skewness_alpha_4,skewness_alpha_5,skewness_alpha_6,skewness_alpha_7,skewness_alpha_8,skewness_alpha_9,...,hjorth_complexity_theta_9,hjorth_complexity_theta_10,hjorth_complexity_theta_11,hjorth_complexity_theta_12,hjorth_complexity_theta_13,hjorth_complexity_theta_14,hjorth_complexity_theta_15,hjorth_complexity_theta_16,hjorth_complexity_theta_17,hjorth_complexity_theta_18
participant,label,,,,,,,,,,,,,,,,,,,,,
0,1,-0.001459,-0.001399,-0.000134,-0.000508,0.000034,-0.000360,0.000066,0.000210,-0.000098,-0.000126,...,1.066221,1.065765,1.066247,1.066939,1.066877,1.066765,1.067738,1.066938,1.067252,1.066341
1,1,-0.000217,0.000180,0.000640,-0.000488,-0.000137,0.000464,-0.000203,0.000137,0.000122,0.000450,...,1.079699,1.074341,1.075017,1.073180,1.072607,1.076331,1.078772,1.075743,1.071450,1.077829
2,1,-0.000077,0.000124,-0.000141,0.000404,-0.000013,0.000864,0.000024,0.000401,0.000041,-0.000387,...,1.079635,1.074006,1.072700,1.074512,1.073340,1.073159,1.081924,1.071899,1.071131,1.073522
3,1,-0.000353,-0.000986,-0.002227,0.001064,-0.001766,-0.001095,-0.000969,-0.002000,-0.001392,-0.002053,...,1.068775,1.061851,1.068365,1.060621,1.067822,1.067072,1.068124,1.068629,1.069859,1.069374
4,1,-0.000266,-0.000957,-0.000271,-0.000307,-0.000687,-0.000770,-0.000590,-0.000392,-0.000707,-0.000348,...,1.069524,1.069628,1.070475,1.065580,1.070224,1.069045,1.069355,1.069126,1.067955,1.068820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2,0.000739,0.000721,0.000610,0.000531,0.000886,0.000867,0.000414,0.000254,0.000452,-0.000041,...,1.066504,1.055693,1.059361,1.054512,1.051394,1.061642,1.063582,1.069470,1.064872,1.066766
84,2,-0.001660,-0.002290,-0.001938,-0.003573,-0.001525,-0.002520,-0.002173,-0.002279,-0.002528,-0.002643,...,1.064891,1.061554,1.063955,1.061491,1.059530,1.064640,1.060608,1.069866,1.069353,1.068360
85,2,-0.003732,0.000306,-0.001269,-0.002465,0.001875,-0.002852,0.000128,0.000273,-0.000574,-0.000384,...,1.071987,1.069672,1.070495,1.070900,1.069904,1.072884,1.072472,1.069260,1.069300,1.070037


In [27]:
cols = list(means.columns)
cols

['skewness_alpha_0',
 'skewness_alpha_1',
 'skewness_alpha_2',
 'skewness_alpha_3',
 'skewness_alpha_4',
 'skewness_alpha_5',
 'skewness_alpha_6',
 'skewness_alpha_7',
 'skewness_alpha_8',
 'skewness_alpha_9',
 'skewness_alpha_10',
 'skewness_alpha_11',
 'skewness_alpha_12',
 'skewness_alpha_13',
 'skewness_alpha_14',
 'skewness_alpha_15',
 'skewness_alpha_16',
 'skewness_alpha_17',
 'skewness_alpha_18',
 'skewness_beta_0',
 'skewness_beta_1',
 'skewness_beta_2',
 'skewness_beta_3',
 'skewness_beta_4',
 'skewness_beta_5',
 'skewness_beta_6',
 'skewness_beta_7',
 'skewness_beta_8',
 'skewness_beta_9',
 'skewness_beta_10',
 'skewness_beta_11',
 'skewness_beta_12',
 'skewness_beta_13',
 'skewness_beta_14',
 'skewness_beta_15',
 'skewness_beta_16',
 'skewness_beta_17',
 'skewness_beta_18',
 'skewness_gamma_0',
 'skewness_gamma_1',
 'skewness_gamma_2',
 'skewness_gamma_3',
 'skewness_gamma_4',
 'skewness_gamma_5',
 'skewness_gamma_6',
 'skewness_gamma_7',
 'skewness_gamma_8',
 'skewness_gam

In [28]:
C_participants = means.loc[means.index.get_level_values('label') == 0].reset_index()
A_participants = means.loc[means.index.get_level_values('label') == 1].reset_index()
B_participants = means.loc[means.index.get_level_values('label') == 2].reset_index()

In [48]:
### set style and figure size
#sns.set(rc = {'figure.figsize':(20,10)})
#sns.set(style = "darkgrid")
#plt.rcParams["figure.autolayout"] = True


for charac in range(10, 11):
    #f, axes = plt.subplots(4, 5)
    #ax_last = axes[3][4]

    col = stop = charac*19
    for i in range(4):
        for j in range(5):
            ### plot last subplot empty
            if col == stop + 19:
                ax_last.set_axis_off()
                break
            one_ch = {'C': list(C_participants[cols[col]]), 'A': list(A_participants[cols[col]]), 'B': list(B_participants[cols[col]])}
            ### make dict same length by adding nan
            max_len = max([len(one_ch['C']), len(one_ch['A']), len(one_ch['B'])])
            for key in one_ch.keys():
                one_ch[key] += [np.nan]*(max_len-len(one_ch[key]))
            one_ch_df = pd.DataFrame(one_ch)

            q1 = one_ch_df.quantile(0.25)
            q3 = one_ch_df.quantile(0.75)
            IQR = q3 - q1
            outliers = one_ch_df[((one_ch_df<(q1-1.5*IQR)) | (one_ch_df>(q3+1.5*IQR)))]
            outlier_ids = get_outlier_ids(outliers, means, 'C')
            print(outlier_ids)
            ### plot the boxplot and stripplot
            #ax = sns.boxplot(data = one_ch_df, ax=axes[i][j])
            #ax = sns.stripplot(data = one_ch_df, color="orange", jitter=0.2, size=2.5, ax=axes[i][j])
            #ax.set_title(cols[col])

            col += 1


    #f.subplots_adjust(hspace=0.2, wspace=0.2)
    #plt.savefig('../BoxPlotsAllFeatures/' + cols[stop][:-len(cols[stop].split('_')[-1])-1] + '_test.png', dpi = 300)
    #plt.show()


[38, 55, 63]
[38, 55, 63]
[38, 55, 63]
[38, 43, 55, 63]
[38, 45, 55, 63]
[38, 45, 55, 63]
[55, 63]
[55, 63]
[63]
[63]
[38, 55, 63]
[38, 55, 63]
[38, 55, 63]
[38, 55, 63]
[63]
[38, 55, 63]
[38, 55, 63]
[45, 55, 63]
[38, 55, 63]


In [30]:
q1 = one_ch_df.quantile(0.25)
q3 = one_ch_df.quantile(0.75)
IQR = q3 - q1
outliers = one_ch_df[((one_ch_df<(q1-1.5*IQR)) | (one_ch_df>(q3+1.5*IQR)))]
outliers

,C,A,B
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,91.538531,NaN,NaN
3,NaN,NaN,45.534406
4,NaN,NaN,NaN
5,NaN,58.318573,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [37]:
list(outliers[~outliers['C'].isna()].index)

[2, 19, 27]

In [41]:
def get_outlier_ids(outliers, means, label):
    lbl_to_int = {'C':0, 'A':1, 'B':2}
    indexes = list(outliers[~outliers[label].isna()].index)

    # convert index to par id
    new_means = means.reset_index()
    start_idx = {}
    for index, row in new_means.iterrows():
        if row['label'] not in start_idx:
            start_idx[row['label']] = index
    
    ids = []
    for idx in indexes:
        ids.append(idx + start_idx[lbl_to_int[label]])
    
    return ids

In [43]:
outlier_ids = get_outlier_ids(outliers, means, 'C')
outlier_ids

[38, 55, 63]